# Grover's search
Author : Dhiraj Madan, Advisory Research Engineer, IBM Research India

 We will learn to implement Grover's search algorithm in this notebook. First lets import all the required libraries

In [2]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
#from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ModuleNotFoundError: No module named 'qiskit'

In [ ]:
from qiskit import assemble
import numpy as np
import matplotlib.pyplot as plt

We will work with 4 qubit data registers. Here we will search for an element amongs 0 to 2^4 -1 =15
We will create an ancilla qubit to store |->. This will be needed to provide a phase.

In [1]:
n=4
grover_ckt = QuantumCircuit(n+1, n)
marked = [1,0,1,1] # 1101 element is marked (lsb to msb)=>13

NameError: name 'QuantumCircuit' is not defined

Here is our oracle function. For a given n (number of data qubits), an integer marked (the index of marked element) and a circuit,
it applies the following oracle $U_{marked}|x>|y> =|x>|y \oplus f(x)>$. 

Here f(x) = 1 if  x is the marked index and 0 otherwise.


In [ ]:
def apply_oracle(n,marked,ckt):
    control0 = [i for i in range(n) if not marked[i]]
    ckt.x(control0)
    ckt.mct(list(range(n)),n)
    ckt.x(control0)
    ckt.draw()
    
    

# Exercise 1
You need to complete the function in the cell below to reflect about the unform superposition state. 
 This should take a circuit object, and an integer n : The number of data qubits 
 (excluding the ancilla qubit to store |-> state). The goal is to add operations to refect about the
 superposed state.
As a step you need to be able to reflect about 0's. You will find the function mct helpful 
https://qiskit.org/documentation/stubs/qiskit.circuit.library.MCMT.html

In [ ]:

def reflect_uniform(ckt,n):
    pass
    




Let's start building the circuit now. We start by create a superposition of all states i.e.
$|\psi> = \frac{1}{\sqrt{2^n}} \sum_{x\in \{0,1\}^n} |x>$. 
Note that the final qubit needs to be placed in state |-> as required.  

In [ ]:
grover_ckt.x(n)
grover_ckt.barrier()
grover_ckt.h(list(range(n+1)))




In [ ]:
grover_ckt.draw()


Let's run this circuit now. We will obtain the statevector and keep the first n qubits (leaving the |-> state ancilla qubit as is).

In [ ]:
svsim = Aer.get_backend('statevector_simulator') # Tell Qiskit how to simulate our circuit
qobj = assemble(grover_ckt)     # Create a Qobj from the circuit for the simulator to run
result = svsim.run(qobj).result() # Do the simulation and return the result
statevector = result.data()['statevector']
statevector = statevector[:2**n]

Let's plot this state vector on a 2 d plane spanned by the marked element state |a> and the superposition of unmarked states |e>. 

In [ ]:
marked = [1,0,1,1] # Corresponds to integer 1101 in binary => 13
ket_a = np.zeros(2**n)
ket_a[13] =1
ket_e = (np.ones(2**n) - ket_a)/np.sqrt(2**n -1)



In [ ]:
def get_projection(psi,e,a ):
    proj =  [np.real(np.vdot(e,psi)), np.real(np.vdot(a,psi))]
    return proj

def plt_vector(proj, axes =[0.0,1.0,0.0,1.0]):
    x_pos = 0
    y_pos = 0
    x_direct = proj[0]
    y_direct = proj[1]

    # Creating plot
    fig, ax = plt.subplots()
    ax.quiver(x_pos, y_pos, x_direct, y_direct,scale=1.0)

    ax.axis(axes)

    # show plot
    plt.show()

In [ ]:
proj = get_projection(statevector, ket_e, ket_a)
plt_vector(proj)

Now lets add the phase oracle which reflects the state about |e>.

In [ ]:
#grover_ckt.append(oracle, list(range(n+1)))
apply_oracle(4,marked,grover_ckt)
grover_ckt.barrier()

This is how the circuit looks now.

In [ ]:
grover_ckt.draw()

Now we reflect about the unform state. We use the function you completed as part of the exercise. 

In [ ]:
reflect_uniform(grover_ckt,n)

In [ ]:
grover_ckt.barrier()
grover_ckt.draw()

Now we run the circuit again and replot the state vector. Observe that it indeed gets rotate colckwise towards the marked state |a>

In [ ]:
svsim = Aer.get_backend('statevector_simulator') # Tell Qiskit how to simulate our circuit
qobj = assemble(grover_ckt)     # Create a Qobj from the circuit for the simulator to run
result = svsim.run(qobj).result() # Do the simulation and return the result
statevector = result.data()['statevector']

statevector = statevector[:2**n]
proj = get_projection(statevector, ket_e, ket_a)
plt_vector(proj)

Lets do another step of phase oracle followed by reflection about uniform.

In [ ]:
apply_oracle(4,marked,grover_ckt)

grover_ckt.barrier()
reflect_uniform(grover_ckt,n)
grover_ckt.barrier()


In [ ]:
grover_ckt.draw()

In [ ]:
svsim = Aer.get_backend('statevector_simulator') # Tell Qiskit how to simulate our circuit
qobj = assemble(grover_ckt)     # Create a Qobj from the circuit for the simulator to run
result = svsim.run(qobj).result() # Do the simulation and return the result
statevector = result.data()['statevector']

statevector = statevector[:2**n]
proj = get_projection(statevector, ket_e, ket_a)
plt_vector(proj)

We observe that it indeed rotates clockwise towards |a>. Lets see when we repeat the 2 steps again.

In [ ]:
apply_oracle(4,marked,grover_ckt)

grover_ckt.barrier()
reflect_uniform(grover_ckt,n)
grover_ckt.barrier()
grover_ckt.draw()


In [ ]:
svsim = Aer.get_backend('statevector_simulator') # Tell Qiskit how to simulate our circuit
qobj = assemble(grover_ckt)     # Create a Qobj from the circuit for the simulator to run
result = svsim.run(qobj).result() # Do the simulation and return the result
statevector = result.data()['statevector']

statevector = statevector[:2**n]
proj = get_projection(statevector, ket_e, ket_a)
plt_vector(proj, axes = [-1.0,1.0,-1.0,1.0])

Now we observe that statevector falls into the second quadrant. At this point further rotations will not help much.

# Exercise 2.1 
Now we compute the theoretical number of rotations which maximize the angle keeping statevector in quadrant I. First compute the initial angle  $\theta_0$. Store it in variable theta0.

In [ ]:
theta0 = 
theta0

# Exercise 2.2
Each rotation increases this by $2 \theta_0$. After T rotations it is $(2T+1)\theta_0$. You need to find the largest such integer T so that it remains in quadrant I. Store it in variable T .

In [ ]:
T = 
T

Now to complete the story we create a new circuit and rotate preciely by number of times given bt T. 

In [ ]:
n=4
grover_ckt = QuantumCircuit(n+1, n)
marked = [1,0,1,1] # 1101 element is marked (lsb to msb)=>13
grover_ckt.x(n)
grover_ckt.barrier()
grover_ckt.h(list(range(n+1)))
grover_ckt.barrier()

for _ in range(int(np.floor(T))):
    apply_oracle(4,marked,grover_ckt)

    grover_ckt.barrier()
    reflect_uniform(grover_ckt,n)
    grover_ckt.barrier()

for j in range(n):
    grover_ckt.measure(j,j)


In [ ]:
grover_ckt.draw()

In [ ]:
sim = Aer.get_backend('qasm_simulator') # Tell Qiskit how to simulate our circuit
qobj = assemble(grover_ckt)     # Create a Qobj from the circuit for the simulator to run
result = sim.run(qobj).result() # Do the simulation and return the result

In [ ]:
result

In [ ]:
counts = result.get_counts(grover_ckt)


In [ ]:
plot_histogram(counts)


# Concluding remarks and points to ponder
We see that the above plot has highest measured probability for marked index i.e. 13 ( or 1101).
This completes the implementation of Grover's search for a single marked element. Here are a few more points to ponder about for future :-
1. What happens when there are more than 1 marked elements. How will the algorithm change. What is the initial angle and rotation in each step.
2. What if you dont know the number of marked elements. What will be the modfications needed ?

# References 
1. L. K. Grover (1996), "A fast quantum mechanical algorithm for database search", Proceedings of the 28th Annual ACM Symposium on the Theory of Computing (STOC 1996), doi:10.1145/237814.237866, arXiv:quant-ph/9605043

1. I. Chuang & M. Nielsen, "Quantum Computation and Quantum Information", Cambridge: Cambridge University Press, 2000.

1. Qiskit Textbook : https://qiskit.org/textbook/ch-algorithms/grover.html
